# Django Tutorial (part 3)
Back the project root:

In [2]:
cd myworkspace/mysite
tree -I __pycache__

bash: cd: myworkspace/mysite: No such file or directory
.
├── db.sqlite3
├── manage.py
├── mysite
│   ├── asgi.py
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── polls
    ├── admin.py
    ├── apps.py
    ├── __init__.py
    ├── migrations
    │   ├── 0001_initial.py
    │   └── __init__.py
    ├── models.py
    ├── tests.py
    ├── urls.py
    └── views.py

3 directories, 16 files


Restart the server

In [3]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

7000/tcp:            64303
[1] 64594


Django has an interactive Python shell which we can use to add data to `sqlite`.  Again, we'll use Unix hereis documents to controll the shell.  Of course, in reality, you would just use the command line.

In [5]:
python manage.py shell << EOF &
from polls.models import Choice, Question  # Import the model classes we just wrote.
print(Question.objects.all())
EOF

<QuerySet []>
[3] 64625
[2]   Done                    python manage.py shell <<EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
print(Question.objects.all())
EOF



We can now create data for a new Question:

In [6]:
python manage.py shell << EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
from django.utils import timezone
q = Question(question_text="What's new?", pub_date=timezone.now())

# Save the object into the database. You have to call save() explicitly.
q.save()
print(q.id)
EOF

<QuerySet []>
1
[3]+  Done                    python manage.py shell <<EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
print(Question.objects.all())
EOF



We can inspect the question text and publication date with a slightly modified script:

In [7]:
# Access model field values via Python attributes.
python manage.py shell << EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
from django.utils import timezone

print(Question.objects.all())
q = Question.objects.all()[0]
print(q)
print(q.question_text)
print(q.pub_date)
EOF

<QuerySet [<Question: Question object (1)>]>
Question object (1)
What's new?
2021-10-07 15:45:35.227973+00:00


That's a bit cryptic.  We can improve the print out by changing the model slightly.
The current model looks like:

In [8]:
cat polls/models.py

from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)


Change the `Question` and `Choice` classes by adding `__str__` methods:

In [9]:
cat << EOF > polls/models.py
from django.db import models
from django.utils import timezone
import datetime

class Question(models.Model):
    def __str__(self):
         return self.question_text
         
    def was_published_recently(self):
         return self.pub_date >= timezone.now() - datetime.timedelta(days=1)

    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')


class Choice(models.Model):
    def __str__(self):
         return self.choice_text

    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
EOF

#python manage.py migrate

/home/chris/home/workspace/python-course/src/37 Django/myworkspace/mysite/polls/models.py changed, reloading.


Now run the queries again (to see text printed):

In [11]:
# Access model field values via Python attributes.
python manage.py shell << EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
from django.utils import timezone

print(Question.objects.all())
q = Question.objects.all()[0]
print(q)
print(q.question_text)
print(q.pub_date)
EOF

<QuerySet [<Question: What's new?>]>
What's new?
What's new?
2021-10-07 15:45:35.227973+00:00


Other things you can do in the shell:

In [12]:
python manage.py shell << EOF
from polls.models import Choice, Question
from django.utils import timezone
print(Question.objects.all())
q = Question.objects.all()[0]
print(q.question_text)
print(q.pub_date)
print(q.id)
print(q.was_published_recently())

print(Question.objects.filter(id=q.id))

EOF

<QuerySet [<Question: What's new?>]>
What's new?
2021-10-07 15:45:35.227973+00:00
1
True
<QuerySet [<Question: What's new?>]>


Give the `Question` a couple of `Choices`.  At present there are no choices:

In [13]:
python manage.py shell << EOF
from polls.models import Choice, Question
q = Question.objects.all()[0]
print(q.choice_set.all())
EOF

<QuerySet []>


Use `create` to create choices:

In [14]:
python manage.py shell << EOF
from polls.models import Choice, Question
# Create three choices.
q = Question.objects.all()[0]
q.choice_set.create(choice_text='Not much', votes=0)
q.choice_set.create(choice_text='The sky', votes=0)
c = q.choice_set.create(choice_text='Just hacking again', votes=0)
print(f"original question: {c.question}")
print(f"choices: {q.choice_set.all()}")
print(f"count of choices: {q.choice_set.count()}")
EOF

original question: What's new?
choices: <QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>]>
count of choices: 3


The API automatically follows relationships as far as you need.
Use double underscores to separate relationships.
This works as many levels deep as you want; there's no limit.

Find all Choices for any question whose pub_date is in this year
(reusing the 'current_year' variable we created above).

In [15]:
python manage.py shell << EOF
from polls.models import Choice, Question
from django.utils import timezone
current_year = timezone.now().year
print(Choice.objects.filter(question__pub_date__year=current_year))


q = Question.objects.all()[0]
c = q.choice_set.filter(choice_text__startswith='Just hacking')
print(q.choice_set.all())
EOF

<QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>]>
<QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>]>


Let's delete one of the choices using delete():

In [16]:
python manage.py shell << EOF
from polls.models import Choice, Question
from django.utils import timezone
current_year = timezone.now().year
q = Question.objects.all()[0]
c = q.choice_set.filter(choice_text__startswith='Just hacking')
c.delete()
print(q.choice_set.all())
EOF

<QuerySet [<Choice: Not much>, <Choice: The sky>]>


We can use a webapp to see all of the above.  First create superuser for Django Admin by
running the following command in xterm:

In [17]:
xterm -fg black -bg white -fa 'Monospace' -fs 14 -e 'python manage.py createsuperuser' &

[2] 64861


Register the `Question` with the app:

In [18]:
cat << EOF > polls/admin.py
from django.contrib import admin
from .models import Question
admin.site.register(Question)

EOF

[2]+  Done                    xterm -fg black -bg white -fa 'Monospace' -fs 14 -e 'python manage.py createsuperuser'


Check the registration worked:

In [20]:
cat polls/admin.py

from django.contrib import admin
from .models import Question
admin.site.register(Question)



Now fire up the admin app and login as the superuser.  You can now investigate all the `Questions` and `Choices`.

In [21]:
firefox http://127.0.0.1:7000/admin/ &

[2] 64909
